## Polytope Climate-DT MN5 example notebook

This notebook shows how to use earthkit-data and earthkit-plots to pull destination-earth data from Mare Nostrum 5 from the `clmn` stream which is used for monthly means and plot it using earthkit-plots.

Before running the notebook you need to set up your credentials. See the main readme of this repository for different ways to do this or use the following cells to authenticate.

You will need to generate your credentials using the desp-authentication.py script.

This can be run as follows:

In [ ]:
%%capture cap
%run ../desp-authentication.py

This will generate a token that can then be used by earthkit and polytope.

In [ ]:
output_1 = cap.stdout.split('}\n')
access_token = output_1[-1][0:-1]

# Requirements
To run this notebook install the following:
* pip install earthkit-data
* pip install earthkit-plots
* pip install earthkit-regrid  (Optional for spectral variables)
* pip install cf-units         (Optional for unit conversion in maps)

If you do not have eccodes installed please install eccodes using conda as it is a dependency of earthkit, or install earthkit via conda

* conda install eccodes -c conda-forge
* conda install earthkit-data -c conda-forge

In [ ]:
import earthkit.data
import earthkit.plots
import earthkit.regrid
from polytope.api import Client

In [ ]:
client = Client(
    address="polytope.mn5.apps.dte.destination-earth.eu")

request = {
    "class": "d1",
    "dataset": "climate-dt",
    "generation": "1",
    "expver": "0001",
    "activity": "scenariomip",
    "experiment": "ssp3-7.0",
    "realization": "1",
    "model": "ifs-fesom",
    "resolution": "standard",
    "stream": "clmn",
    "type": "fc",
    "year": "2020",
    "month": "2",
    "levtype": "sfc",
    "param": "228004"
}

file = client.retrieve("destination-earth", request, "data/climate-dt-earthkit-MN5-monthly-mean-example.grib")

In [ ]:
data = earthkit.data.from_source("file", "data/climate-dt-earthkit-MN5-monthly-mean-example.grib")

In [ ]:
data.ls()

In [ ]:
chart = earthkit.plots.Map(extent=[-180, 180, -90, 90])
chart.contourf(
    data[0],
    auto_style=True,
    units="kelvin"
)

chart.title("Average Monthly 2 Meter Temperature (K)")
chart.coastlines()
chart.gridlines()
chart.show()

In [ ]:
# Regrid t=from healpix for conversion to xarray
data_latlon = earthkit.regrid.interpolate(data, out_grid={"grid": [1,1]}, method="linear")
# Convert data to xarray
data_latlon.to_xarray()